# Testing Torch Choice Package

The purpose of this notebook is to test out the capabilities of the torch-choice package in python. The first section will try the vignette and then some new data will be used.

## Vignette


Install required packages

In [ ]:
import pandas as pd
import numpy as np
from torch_choice.utils.easy_data_wrapper import EasyDatasetWrapper

In [ ]:
car_choice = pd.read_csv("https://raw.githubusercontent.com/gsbDBI/torch-choice/main/tutorials/public_datasets/car_choice.csv")
